## Initalize notebook and load configuration

In [1]:
import esf_utils
import os 
import pandas as pd 
import xml.etree.ElementTree as ET
import shutil
import configparser
import time
import datetime
import sys

print ( 'dependencies loaded.')

campaign_esf_paths_file = 'turn_one.txt'
campaign_files = esf_utils.parse_campaign_files_txt( campaign_esf_paths_file )


config = configparser.ConfigParser()
config.read('config.ini')

esf2xml_dir =  config['dependencies']['esf2xml']
save_folder = config['paths']['save_game_folder']
output_folder = config['paths']['output_folder']
extracted_subfolder = config['paths']['extracted_subfolder']
cwd = os.getcwd()

print('notebook initialized')

dependencies loaded.
loading esf paths from file @  turn_one.txt
notebook initialized


## Testing all together 

In [18]:
# notebook for parsing a file 
econ_array = []
army_array = [] 
region_array = []

max = len(campaign_files)-1
i = 0 

for s in campaign_files:

    save_file_clean = esf_utils.clean_filename( s )
    out = output_folder
    templated = out.replace( '[$filename]', save_file_clean )
    
    full_path = os.path.join( save_folder , s )
    #output_dir =  f"{save_file_clean}_extract"
    file_modstamp = os.path.getmtime(full_path)
    ts = datetime.datetime.fromtimestamp( file_modstamp )
    unix_timestamp = int(time.mktime(ts.timetuple()))

    try:
        dat1 = esf_utils.extract_save_file( 
        save_folder 
        , s
        ,templated
        ,config 
        )
    except:
        print("An exception occurred while extracting")
    

    extracted_output = os.path.join(templated, extracted_subfolder)
    
    try :
        session_id = esf_utils.get_session_guid( extracted_output )[1]
        turn_num = esf_utils.get_turn_number( extracted_output )
    
        # for faction economics at a high KPI level
        new_array = []
        esf_utils.parse_extracted_factions_folder( extracted_output, new_array ) 
        for r in new_array:
            r["session_id"] = session_id
            r["turn_num"] = turn_num
            r["modifiedOn"] = unix_timestamp
            econ_array.append( r ) 
        econ_df = pd.DataFrame( econ_array )
        econ_df.to_csv('export_faction_economy.csv')
        
        # parse army information
        new_array = []
        #session_id, session_guid, turn_num = 
        esf_utils.parse_extracted_armies_folder( extracted_output, new_array ) 

        # write in all data universal to this save file
        for r in new_array:
            r["session_id"] = session_id
            r["turn_num"] = turn_num
            r["modifiedOn"] = unix_timestamp
            army_array.append( r )
        army_df = pd.DataFrame( army_array )
        army_df.to_csv('export_army_unit.csv')

        # parse region information
        new_array = []
        esf_utils.parse_extracted_region_folder( extracted_output, new_array ) 
        
         # write in all data universal to this save file
        for r in new_array:
            r["session_id"] = session_id
            r["turn_num"] = turn_num
            r["modifiedOn"] = unix_timestamp
            region_array.append( r )
        region_df = pd.DataFrame( region_array ) 
        region_df.to_csv( 'export_region.csv' )
        
        # first run - get province hierarchy 
        if i == 0 :
            print ( 'parsing province xml data')
            new_array = []
            esf_utils.parse_province_campaign_data( extracted_output , new_array )
            province_df = pd.DataFrame( new_array )
            province_df.to_csv( 'export_province.csv')
    except:
        print("An exception occurred while extracting")
   
    print( f"@ { i }  / { max } folders loaded and exported")
    i += 1

skipping extract...
getting economy data from  extract\Clan_Skryre_Auto_save.181597358761.save_extract\extract\
An exception occurred while extracting
@ 0  / 1 folders loaded and exported
need to convert to .esf for esf2xml to work, copying file....
 copy D:\TW2_SAVEGAME_BACKUPS\Clan Skryre_Auto-save.191597358761.save -> c:\lab\tw_save\Clan_Skryre_Auto_save.191597358761.esf
jruby C:\lab\etwng-master\esfxml\esf2xml c:\lab\tw_save\Clan_Skryre_Auto_save.191597358761.esf c:\lab\tw_save\extract\Clan_Skryre_Auto_save.191597358761.save_extract\
jruby C:\lab\etwng-master\esfxml\esf2xml c:\lab\tw_save\extract\Clan_Skryre_Auto_save.191597358761.save_extract\compressed_data.esf c:\lab\tw_save\extract\Clan_Skryre_Auto_save.191597358761.save_extract\extract\


# Just testing extract_regions  

In [ ]:
for s in campaign_files:
    save_file_clean = esf_utils.clean_filename( s )
    out = output_folder
    templated = out.replace( '[$filename]', save_file_clean )
    full_path = os.path.join( save_folder , s )   
    extracted_output = os.path.join(templated, extracted_subfolder)

    # for region and control 
    new_array = []
    esf_utils.parse_extracted_region_folder( extracted_output, new_array ) 
    region_df = pd.DataFrame( new_array )
    region_df.to_csv( 'export_region.csv' , index=False )
    print ( 'exported to region.csv')   
 

In [ ]:
## Just provinces 


def parse_province_campaign_data( extracted_output, new_array ):
    full_path = os.path.join( extracted_output , 'campaign_env' , 'world-0000.xml' ) 
    xml = ET.parse( full_path ).getroot()    

    # get name NK of region from file, NOT filename 
    province_xml = xml.findall( "./rec/ary/rec[@type = 'PROVINCE_ARRAY']")

    xml_index = 0 
    for province in province_xml: 

        #if xml_index == 0 :
        # parse for testing 
        province_name = province.findall( './asc')[0].text 
        #print ( 'province name is ' , province_name ) 

        region_blocks = province.findall( "./rec/rec/ary/rec/asc" )
        #print ( 'length of region blocks ', len( region_blocks ) )
        for region in region_blocks : 
            data_row = {
                'province_name' : province_name
                ,'settlement_key' : region.text
            }
            new_array.append( data_row )
    xml_index = xml_index + 1

for s in campaign_files:
    save_file_clean = esf_utils.clean_filename( s )
    out = output_folder
    templated = out.replace( '[$filename]', save_file_clean )
    full_path = os.path.join( save_folder , s )   
    extracted_output = os.path.join(templated, extracted_subfolder)

    # for characters
    new_array = []
    esf_utils.parse_extracted_region_folder( extracted_output, new_array ) 
    region_df = pd.DataFrame( new_array )
    region_df.to_csv( 'export_region.csv' , index=False )
    print ( 'exported to region.csv')   
 

In [12]:
## Just character 


def parse_character_campaign_data( extracted_output, new_array ):

    print( 'getting character data from ', extracted_output )

    #print ( 'debugger')
    # get factions data
    character_dir = os.path.join( extracted_output , "character" )


    arr = os.listdir( character_dir )
    for file in os.listdir( character_dir ):
        #print('file', file )
        if file.endswith(".xml"):
            if file.find("general") != -1:
                # parse_faction_xml( factions_dir , file, data_array )
                
                full_path = os.path.join( character_dir , file ) 
                print(full_path )
                xml = ET.parse( full_path ).getroot()
                
                coord_xpath = "./rec/rec[@type = 'LOCOMOTABLE']/v2"   
                coords = xml.findall( coord_xpath )[0]

                x = coords.attrib['x']
                y = coords.attrib['y']
                print( coords , x ,y ) 
                
                details_xpath = "./rec/rec[@type = 'CHARACTER_DETAILS']/asc"   
                detail_tags = xml.findall( details_xpath )
                faction = detail_tags[0].text 
                type = detail_tags[2].text 
                key = detail_tags[3].text

                data_row = {
                    'faction' : faction
                    ,'type' : type
                    ,'key' : key 
                    , 'loc.x' : x
                    , 'loc.y' : y  
                }
                new_array.append( data_row )
                print('debugger')

                '''
                campaign_tags = mf_xml.findall( "./rec/rec/[@type = 'LOCOMOTABLE']" )[0] 
                localized = campaign_tags.findall( "asc")
                unit_campaign_name = localized[0].text

                if ( unit_campaign_name is None ):
                    unit_campaign_name = 'null'

                '''

for s in campaign_files:
    save_file_clean = esf_utils.clean_filename( s )
    out = output_folder
    templated = out.replace( '[$filename]', save_file_clean )
    full_path = os.path.join( save_folder , s )   
    extracted_output = os.path.join(templated, extracted_subfolder)

    # for characters
    new_array = []
    parse_character_campaign_data( extracted_output , new_array )

    
    #esf_utils.parse_extracted_region_folder( extracted_output, new_array ) 
    characters_df = pd.DataFrame( new_array )
    region_df.to_csv( 'export_region.csv' , index=False )
    print ( 'exported to region.csv')   
 

getting character data from  extract\turn1_eatine_noMove.1349987008.save_extract\extract\
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0001.xml
<Element 'v2' at 0x0000023E4B9A4680> 140.37932 211.49678
debugger
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0002.xml
<Element 'v2' at 0x0000023E4B90FEA0> 158.42809 62.908714294433594
debugger
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0003.xml
<Element 'v2' at 0x0000023E4B9C4C20> 151.07489 278.65088
debugger
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0004.xml
<Element 'v2' at 0x0000023E4B9C4810> 64.1734 302.57935
debugger
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0005.xml
<Element 'v2' at 0x0000023E4B9D6860> 193.85715 265.52881
debugger
extract\turn1_eatine_noMove.1349987008.save_extract\extract\character\general-0006.xml
<Element 'v2' at 0x0000023E4B9C1680> 187.8409 268.2